In [6]:
!pip install torch
!pip install torch-geometric
!pip install scikit-learn numpy
!pip install torch torch-geometric transformers scikit-learn

In [8]:
import torch
import torch.nn.functional as F
import numpy as np
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Usando dispositivo: {device}")

Usando dispositivo: cpu


In [10]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
bert_model = AutoModel.from_pretrained('bert-base-uncased').to(device)

#função original para obter embeddings de texto
def get_text_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128).to(device)
    with torch.no_grad():
        outputs = bert_model(**inputs)
#usamos a média dos tokens da última camada oculta
    return outputs.last_hidden_state.mean(dim=1).squeeze()

#bert-base-uncased retorna embeddings de 768 dimensões
EMBEDDING_DIM = 768

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [11]:
#dados de entrada teste1
users = [
    {"name": "Ana", "preference": "Gosta de lugares artísticos e café"},
    {"name": "Bruno", "preference": "Prefere cerveja artesanal e música ao vivo"}
]

places = [
    {"name": "Café Arte & Som", "desc": "Café acolhedor com Wi-Fi, vibe artística e eventos musicais"},
    {"name": "Bar do Zé", "desc": "Bar descontraído com cervejas artesanais e música ao vivo"},
    {"name": "Espaço Cultural Vila", "desc": "Espaço para exposições e workshops de arte"},
]

In [12]:
#embeddings de conteudo (do seu MVP)
user_embeddings_initial = [get_text_embedding(u['preference']) for u in users]
place_embeddings_initial = [get_text_embedding(p['desc']) for p in places]

#criando matriz de features (X) do grafo
#[Ana(0), Bruno(1), Café Arte(2), Bar do Zé(3), Espaço Cultural(4)]
initial_features_x = torch.stack(user_embeddings_initial + place_embeddings_initial).to(device)

print("Matriz de Features Iniciais (X) criada com BERT:")
print(initial_features_x.shape)

Matriz de Features Iniciais (X) criada com BERT:
torch.Size([5, 768])


In [14]:
edge_index = torch.tensor([
    [0, 0, 1, 1],
    [2, 4, 3, 2]
], dtype=torch.long).to(device)

graph_data = Data(x=initial_features_x, edge_index=edge_index)

print("\nObjeto do Grafo:")
print(graph_data)


Objeto do Grafo:
Data(x=[5, 768], edge_index=[2, 4])


In [16]:
#GNN modelo mais simples
class GNNRecommender(torch.nn.Module):
    def __init__(self, num_features, hidden_dim):
        super().__init__()
        self.conv1 = GCNConv(num_features, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)

        return x

In [17]:
HIDDEN_DIM = 128
model = GNNRecommender(num_features=EMBEDDING_DIM, hidden_dim=HIDDEN_DIM).to(device)

model.eval()
print(f"Modelo GNNRecommender criado. Dimensão de saída: {HIDDEN_DIM}")

Modelo GNNRecommender criado. Dimensão de saída: 128


In [18]:
with torch.no_grad():
    final_embeddings = model(graph_data)

print("\nShape dos Embeddings Finais (enriquecidos pelo grafo):")
#2 usuários, 3 lugares), cada um com 128 dimensões
print(final_embeddings.shape)


Shape dos Embeddings Finais (enriquecidos pelo grafo):
torch.Size([5, 128])


In [22]:
final_user_embeddings = final_embeddings[0:len(users)]
final_place_embeddings = final_embeddings[len(users):]

user_preference_text = "Bares descontraídos e eventos sociais"
# Usamos a "Ana" (usuário 0) como exemplo, pois sua preferência é artística
# user_preference_text = users[0]['preference']

print(f"--- Recomendação GNN para a preferência: '{user_preference_text}' ---")

user_preference_initial_embedding = get_text_embedding(user_preference_text).to(device)

user_preference_graph_data = Data(x=user_preference_initial_embedding.unsqueeze(0), edge_index=torch.empty((2, 0), dtype=torch.long).to(device))

with torch.no_grad():
    user_preference_vector = model(user_preference_graph_data).cpu().numpy().reshape(1, -1)


# 2. Pegamos os embeddings dos LUGARES (gerados pela GNN)
place_vectors = final_place_embeddings.cpu().detach().numpy()

similarities = cosine_similarity(user_preference_vector, place_vectors)[0]

# 4. Encontramos a melhor recomendação
top_place_idx = np.argmax(similarities)
recommended_place = places[top_place_idx]

print(f"\nLugar Recomendado: {recommended_place['name']}")
print(f"Descrição: {recommended_place['desc']}")

--- Recomendação GNN para a preferência: 'Bares descontraídos e eventos sociais' ---

Lugar Recomendado: Espaço Cultural Vila
Descrição: Espaço para exposições e workshops de arte
